In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pretty_midi

In [2]:
MODEL_PATH = "../models/music_generator.h5"
ARTIFACT_DIR = "artifacts"
OUTPUT_DIR = "../output/generated_music"

MOODS = ["happy", "sad", "angry", "quiet"]
SEQ_LEN = 50
GENERATE_LENGTH = 300   # number of notes to generate

In [3]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
model = load_model(MODEL_PATH)

with open(f"{ARTIFACT_DIR}/note_to_idx.pkl", "rb") as f:
    note_to_idx = pickle.load(f)

idx_to_note = {i: n for n, i in note_to_idx.items()}
vocab_size = len(note_to_idx)

print("Model loaded ")
print("Vocabulary size:", vocab_size)

Model loaded 
Vocabulary size: 128


In [5]:
def generate_seed_sequence():
    return np.random.randint(0, vocab_size, size=(SEQ_LEN,))

In [6]:
def generate_music(mood_name):
    mood_id = MOODS.index(mood_name)

    sequence = generate_seed_sequence().tolist()
    generated = []

    for _ in range(GENERATE_LENGTH):
        input_seq = np.array(sequence[-SEQ_LEN:]).reshape(1, SEQ_LEN)
        mood_input = np.array([[mood_id]])

        preds = model.predict([input_seq, mood_input], verbose=0)[0]
        next_note = np.random.choice(len(preds), p=preds)

        generated.append(next_note)
        sequence.append(next_note)

    return generated

In [7]:
def notes_to_midi(notes, file_name):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program("Acoustic Grand Piano"))

    start = 0
    duration = 0.3

    for note_idx in notes:
        pitch = int(idx_to_note[note_idx])
        note = pretty_midi.Note(
            velocity=100,
            pitch=pitch,
            start=start,
            end=start + duration
        )
        instrument.notes.append(note)
        start += duration

    midi.instruments.append(instrument)
    midi.write(file_name)

In [12]:
mood = "happy"   # change mood here

generated_notes = generate_music(mood)

output_path = f"{OUTPUT_DIR}/{mood}_generated.midi"
notes_to_midi(generated_notes, output_path)

print(f"Music generated for mood '{mood}'")
print(f"Saved at: {output_path}")

Music generated for mood 'happy'
Saved at: ../output/generated_music/happy_generated.midi
